In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
teis_p = pd.read_csv('../data/teis_p.csv')
teis_p.info()

In [ ]:
teis_c = pd.read_csv('../data/teis_c.csv')
teis_c.info()

In [ ]:
teis_c = teis_c.rename(columns = {'Notification/ Referral Date' : 'Notification Date'})
teis_p = teis_p.rename(columns = {'County SES ' : 'County SES'})

The next group of cells is experimenting with different ways of joining the two datasets, looking at duplicates, etc.

In [ ]:
teis_merge = pd.merge(teis_c, teis_p, how='outer')
teis_merge.info()

In [ ]:
full_teis= pd.concat([teis_c, teis_p])

In [ ]:
teis_clean = full_teis[~full_teis.duplicated(['POE', 'Child ID', 'DOB', 'County Name', 'County SES', 'Child Phase',
       'Active', 'Service Coordinator', 'Notification Date',
       'Parent Consent Date', 'Referral Source Type Name',
       'Initial Eligibility', 'Initial Eligibility Date', 'Initial IFSP Date',
       'Latest IFSP Date', 'Exit Reason', 'Exit Date'])]
teis_clean.info()
#pulling out rows that are the same across all columns that the two dataframes share

In [ ]:
import re
from tqdm.notebook import tqdm

In [ ]:
for ind, row in tqdm(teis_clean.iterrows()):
    if re.search('\d{5}', str(row['Exit Date'])):
        teis_clean.loc[ind, 'Exit Date'] = np.NaN
#this is clearing out the weird data that's in the 'Exit Date' column

In [ ]:
teis_clean[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']] = teis_clean[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']].apply(pd.to_datetime)
teis_clean

In [ ]:
teis_clean = teis_clean.drop(columns = ['2012 Child Count', '2013 Child Count', 'Service Coordinator Counter', 'Referral Count',
                                        'Evaluation Count', 'Eligibility Count', 'IFSP Count', 'Child Count', ' QTR ', 
                                       'Referral Source Category'])
teis_clean.head(2)
#there isn't meaningful data in these columns (at least not for the purposes of this project)

In [ ]:
all_teis_dups = teis_clean[teis_clean.duplicated(['Child ID', 'Notification Date'], keep=False)].sort_values('Child ID')
all_teis_dups
#pulling all rows that have 'Child ID' and 'Notification Date' in common to take a look

In [ ]:
num_all_dup = len(np.unique(np.array(all_teis_dups['Child ID'])))
num_all_dup

In [ ]:
county_dups = all_teis_dups[~all_teis_dups.duplicated(['County Name', 'Child ID', 'Notification Date'], keep=False)]
county_dups
#These are instances when the same referral was noted under 2 different counties

In [ ]:
num_county_dup = len(np.unique(np.array(county_dups['Child ID'])))
num_county_dup

In [ ]:
POE_dups = all_teis_dups[~all_teis_dups.duplicated(['POE', 'Child ID', 'Notification Date'], keep=False)]
POE_dups
#These are instances when the same referral was noted under 2 different POEs

In [ ]:
county_status_dups = county_dups[county_dups.duplicated(['Child ID', 'Child Phase', 'Notification Date'], keep=False)]
county_status_dups
#looking at the duplicates that have different counties but the same status

In [ ]:
teis_c[teis_c.duplicated(['Child ID', 'Notification Date'], keep=False)]
#reconfirming that this original dataframe doesn't have any rows that have the same Child ID and Notification Date

#### Notes on our decisions regarding how to most effectively merge our data
After exploring the duplicate data, we decided to drop all columns from the teis_c dataframe that it shares with the teis_p dataframe except for Child ID and Notification Date. This will allow us to merge without creating any duplicates, keep the "extra" columns in teis_c, and per the folks at TEIS, the data from the teis_p dataframe should supercede any from the teis_c dataframe because it is more current

In [ ]:
teis_c2 = teis_c.drop(columns = ['POE', 'DOB', 'County Name', 'County SES', 'Child Phase', 'Active', 'Service Coordinator', 
                                 'Parent Consent Date', 'Referral Source Type Name', 'Initial Eligibility', 'Initial Eligibility Date', 'Initial IFSP Date', 'Latest IFSP Date', 'Exit Reason', 'Exit Date',
                                 '2012 Child Count', '2013 Child Count', 'Service Coordinator Counter', 'Referral Count', 
                                 'Evaluation Count', 'Eligibility Count', 'IFSP Count', 'Child Count', ' QTR ', 
                                 'Referral Source Category'])
teis_c2

In [ ]:
teis_merge3 = pd.merge(teis_c2, teis_p, how='outer').sort_values(['Child ID'])
teis_merge3

In [ ]:
teis_merge3[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']] = teis_merge3[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']].apply(pd.to_datetime)
teis_merge3.info()

In [ ]:
teis_c3 = teis_c.drop(columns = ['2012 Child Count', '2013 Child Count', 'Service Coordinator Counter', 'Referral Count', 
                                 'Evaluation Count', 'Eligibility Count', 'IFSP Count', 'Child Count', ' QTR ', 
                                 'Referral Source Category'])
teis_c3

In [ ]:
for ind, row in tqdm(teis_c3.iterrows()):
    if re.search('\d{5}', str(row['Exit Date'])):
        teis_c3.loc[ind, 'Exit Date'] = np.NaN

In [ ]:
teis_c3[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']] = teis_c3[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']].apply(pd.to_datetime)
teis_c3

In [ ]:
teis_merge2 = pd.merge(teis_merge3, teis_c3, on=['Child ID', 'Notification Date'], how='left')
teis_merge2.info()
##Need to do this to bring back in the data for the rows in our merge that were present only in teis_c

In [ ]:
teis_merge2['Referral Source Type Name_x'] = teis_merge2['Referral Source Type Name_x'].fillna(teis_merge2['Referral Source Type Name_y'])
teis_merge2['Late Referral_x'] = teis_merge2['Late Referral_x'].fillna(teis_merge2['Late Referral_y'])
teis_merge2['Qtr_x'] = teis_merge2['Qtr_x'].fillna(teis_merge2['Qtr_y'])
teis_merge2['POE_x'] = teis_merge2['POE_x'].fillna(teis_merge2['POE_y'])
teis_merge2['DOB_x'] = teis_merge2['DOB_x'].fillna(teis_merge2['DOB_y'])
teis_merge2['County Name_x'] = teis_merge2['County Name_x'].fillna(teis_merge2['County Name_y'])
teis_merge2['County SES_x'] = teis_merge2['County SES_x'].fillna(teis_merge2['County SES_y'])
teis_merge2['Child Phase_x'] = teis_merge2['Child Phase_x'].fillna(teis_merge2['Child Phase_y'])
teis_merge2['Service Coordinator_x'] = teis_merge2['Service Coordinator_x'].fillna(teis_merge2['Service Coordinator_y'])
teis_merge2['Parent Consent Date_x'] = teis_merge2['Parent Consent Date_x'].fillna(teis_merge2['Parent Consent Date_y'])
teis_merge2['Initial Eligibility_x'] = teis_merge2['Initial Eligibility_x'].fillna(teis_merge2['Initial Eligibility_y'])
teis_merge2['Initial Eligibility Date_x'] = teis_merge2['Initial Eligibility Date_x'].fillna(teis_merge2['Initial Eligibility Date_y'])
teis_merge2['Initial IFSP Date_x'] = teis_merge2['Initial IFSP Date_x'].fillna(teis_merge2['Initial IFSP Date_y'])
teis_merge2['Latest IFSP Date_x'] = teis_merge2['Latest IFSP Date_x'].fillna(teis_merge2['Latest IFSP Date_y'])
teis_merge2['Exit Reason_x'] = teis_merge2['Exit Reason_x'].fillna(teis_merge2['Exit Reason_y'])
teis_merge2['Exit Date_x'] = teis_merge2['Exit Date_x'].fillna(teis_merge2['Exit Date_x'])
teis_merge2.info()
#pulling data from the teis_c3 dataframe into the columns that originated from the teis_merge3 dataframe but had null values

In [ ]:
teis_merge2 = teis_merge2.drop(columns = ['POE_y', 'DOB_y', 'County Name_y', 'County SES_y', 'Child Phase_y', 'Active_y',
                                          'Service Coordinator_y', 'Parent Consent Date_y', 'Referral Source Type Name_y', 
                                          'Initial Eligibility_y', 'Initial Eligibility Date_y', 'Initial IFSP Date_y',
                                          'Latest IFSP Date_y', 'Exit Reason_y', 'Exit Date_y', 'Fiscal Year_y',
                                          'Notification Month_y', 'Tenn Region_y', 'Fiscal Year.1_y', 'third DOB_y', 
                                          'Late Referral_y', 'Qtr_y'])
teis_merge2.info()
#dropping the no longer needed columns now that we have the cells needed

In [ ]:
teis_merge2 = teis_merge2.rename(columns= {'Fiscal Year_x': 'Fiscal Year',
                                           'Notification Month_x' : 'Notification Month', 
                                           'Tenn Region_x' : 'Tenn Region', 
                                           'Fiscal Year.1_x' : 'Fiscal Year.1',
                                           'third DOB_x' : 'third_DOB',
                                           'Late Referral_x' : 'Late Referral',
                                           'Qtr_x' : 'Qtr',
                                           'POE_x' : 'POE',
                                           'DOB_x' : 'DOB',
                                           'County Name_x' : 'County Name',
                                           'County SES_x' : 'County SES',
                                           'Child Phase_x' : 'Child Phase',
                                           'Active_x' : 'Active',
                                           'Service Coordinator_x' : 'Service Coordinator',
                                           'Parent Consent Date_x' : 'Parent Consent',
                                           'Referral Source Type Name_x' : 'Referral Source Type Name',
                                           'Initial Eligibility_x' : 'Initial Eligibility',
                                           'Initial Eligibility Date_x' : 'Initial Eligibility Date',
                                           'Initial IFSP Date_x' : 'Initial IFSP Date',
                                           'Latest IFSP Date_x' : 'Latest IFSP',
                                           'Exit Reason_x' : 'Exit Reason',
                                           'Exit Date_x' : 'Exit Date'})
teis_merge2.info()
#renaming columns to get rid of the '_x'

In [ ]:
teis_merge2['Year'] = teis_merge2['Notification Date'].dt.year
teis_merge2.head(2)

In [ ]:
all_dups2 = teis_merge2[teis_merge2.duplicated(['Child ID', 'Notification Date'], keep=False)].sort_values('Child ID')
all_dups2
#making sure that there aren't any unanticipated duplicates

In [ ]:
teis_merge2['Referral Age'] = teis_merge2['Notification Date']-teis_merge2['DOB']
teis_merge2['Exit Age'] = teis_merge2['Exit Date']-teis_merge2['DOB']
teis_merge2 = teis_merge2.rename(columns ={'Year' : 'Notification Year'})
teis_merge2

In [ ]:
teis_merge2 = teis_merge2[['Child ID', 'DOB', 'Notification Date', 'Referral Age', 'Referral Source Type Name', 'Child Phase',
                           'Exit Reason', 'Exit Date', 'Exit Age', 'Service Coordinator', 'Parent Consent', 'Initial Eligibility', 
                           'Initial Eligibility Date', 'Initial IFSP Date', 'Latest IFSP', 'Late Referral', 'third_DOB', 'POE',
                           'County Name', 'County SES', 'Tenn Region', 'Fiscal Year', 'Fiscal Year.1', 'Notification Month','Qtr',
                           'Notification Year', 'Active']]
teis_merge2

In [ ]:
#POE_svc_abavg.to_excel('../data/Service_Coordinators_Above_Avg_Early_Exit_Rate.xlsx')

### Pickle File
   

import pickle
#to create a pickle file:
with open('../data/teis_final_data.pickle', 'wb') as file:
    pickle.dump(teis_merge2, file)
#wb means write bytes, aka write a file
#rb means read bytes

#to read in a pickle file:
with open('../data/teis_final_data.pickle', 'rb') as file:
    final_data = pickle.load(file)

final_data

### Bullet 1
    -What is the rate of re-referral? 
    -How many children have multiple referrals (by year or by Point of Entry Office (POE))?

In [ ]:
reref = teis_merge2[teis_merge2.duplicated('Child ID')].sort_values('Child ID')
reref.info()
#pulling out all instances when a child had a second, third, etc. referral

In [ ]:
num_reref = len(np.unique(np.array(reref['Child ID'])))
num_reref
#number of children who had 2 or more referrals

In [ ]:
refsou_reref = reref.groupby(['Referral Source Type Name'])['Child ID'].count()
refsou_reref = refsou_reref.to_frame().reset_index()
refsou_reref = refsou_reref.rename(columns = {'Child ID' : 'Total Re-referrals'})
refsou_reref
#Child ID count of re-referrals by Referral Source Type Name

In [ ]:
refsou_ref = teis_merge2.groupby(['Referral Source Type Name'])['Child ID'].count()
refsou_ref = refsou_ref.to_frame().reset_index()
refsou_ref = refsou_ref.rename(columns = {'Child ID' : 'Total Referrals'})
refsou_ref
#Child ID count of re-referrals by Referral Source Type Name

In [ ]:
refsou_rerate = pd.merge(refsou_reref, refsou_ref, how='left').fillna(0)
refsou_rerate['Re-referral Rate'] = (refsou_rerate['Total Re-referrals'] / refsou_rerate['Total Referrals'])*100
refsou_rerate['Proportion of Referrals'] = (refsou_rerate['Total Referrals'] / refsou_rerate['Total Referrals'].sum()) * 100
refsou_rerate['Proportion of Re-referrals'] = (refsou_rerate['Total Re-referrals'] / refsou_rerate['Total Re-referrals'].sum()) * 100
refsou_rerate

In [ ]:
poe_reref = reref.groupby(['POE'])['Child ID'].count()
poe_reref = poe_reref.to_frame().reset_index()
poe_reref = poe_reref.rename(columns = {'Child ID' : 'Total Re-referrals'})
poe_reref
#Child ID count of re-referrals by Point of Entry

In [ ]:
poe_ref = teis_merge2.groupby(['POE'])['Child ID'].count()
poe_ref = poe_ref.to_frame().reset_index()
poe_ref = poe_ref.rename(columns = {'Child ID' : 'Total Referrals'})
poe_ref
#Child ID count of re-referrals by Point of Entry

In [ ]:
poe_rerate = pd.merge(poe_ref, poe_reref, how='left').fillna(0)
poe_rerate['Re-referral Rate'] = (poe_rerate['Total Re-referrals'] / poe_rerate['Total Referrals'])*100
poe_rerate

In [ ]:
yr_reref = reref.groupby(['Notification Year'])['Child ID'].count()
yr_reref = yr_reref.to_frame().reset_index()
yr_reref = yr_reref.rename(columns = {'Child ID' : 'Total Re-referrals'})
yr_reref
#Child ID count of re-referrals by Year

In [ ]:
yr_ref = teis_merge2.groupby(['Notification Year'])['Child ID'].count()
yr_ref = yr_ref.to_frame().reset_index()
yr_ref = yr_ref.rename(columns = {'Child ID' : 'Total Referrals'})
yr_ref
#Child ID count of re-referrals by Year

In [ ]:
yr_rerate = pd.merge(yr_ref, yr_reref, how='left').fillna(0)
yr_rerate['Re-referral Rate'] = (yr_rerate['Total Re-referrals'] / yr_rerate['Total Referrals'])*100
yr_rerate

In [ ]:
poey_reref = reref.groupby(['POE', 'Notification Year'])['Child ID'].count()
poey_reref = poey_reref.to_frame().reset_index()
poey_reref = poey_reref.rename(columns = {'Child ID' : 'Total Re-referrals'})
poey_reref

In [ ]:
poey_ref = teis_merge2.groupby(['POE', 'Notification Year'])['Child ID'].count()
poey_ref = poey_ref.to_frame().reset_index()
poey_ref = poey_ref.rename(columns = {'Child ID' : 'Total Referrals'})
poey_ref

In [ ]:
poey_rerate = pd.merge(poey_ref, poey_reref, how='left').fillna(0)
poey_rerate['Re-referral Rate'] = (poey_rerate['Total Re-referrals'] / poey_rerate['Total Referrals'])*100
poey_rerate

In [ ]:
ti = sns.FacetGrid(poey_rerate, col="POE", col_wrap=3, hue="POE", palette = 'CMRmap_r')
ti.map_dataframe(sns.lineplot, x="Notification Year", y="Re-referral Rate")
ti.set_axis_labels("Year", "Re-referral Rate");

In [ ]:
tij = sns.lineplot(x="Notification Year", y="Re-referral Rate", data=poey_rerate, hue="POE", palette = 'CMRmap_r')
tij.figure.set_size_inches(14, 10);

#### POE List
    - FT: First Tennessee
    - UC: Upper Cumberland
    - NW Northwest
    - ET: East Tennessee
    - GN: Greater Nashville
    - SW: Southwest
    - SE: Southeast
    - SC: South Central
    - MD: Memphis Delta

### Bullet 2
A referral is considered a "successful" referral if it results in an Individualized Family Service Plan (IFSP). 

    -Are there referral sources that have had higher rates of successful referrals? 

You can determine whether a referral was successful by looking at whether the Initial IFSP Date column is filled in or whether the Child Phase column is equal to IFSP.

In [ ]:
ifsp = teis_merge2.loc[teis_merge2['Child Phase'] == 'IFSP']
ifsp.info()
#children who reach the phase of 'IFSP'

In [ ]:
refsou_suc = ifsp.groupby(['Referral Source Type Name'])['Child ID'].count()
refsou_suc = refsou_suc.to_frame().reset_index()
refsou_suc = refsou_suc.rename(columns = {'Child ID' : 'Total Successful Referrals'})
refsou_suc
#Child ID count for successful referrals by referral source (as defined by reaching IFSP phase)

In [ ]:
refsou_sucrate = pd.merge(refsou_ref, refsou_suc, how='left').fillna(0)
refsou_sucrate['Success Rate'] = (refsou_sucrate['Total Successful Referrals'] / refsou_sucrate['Total Referrals'])*100
refsou_sucrate

### Bullet 3
Some areas of potential concern are POEs having a:

        -low proportion of referrals, 
        -a high average age of referrals, 
        -a low percent of total referrals resulting in an IFSP or 
        -a lower percent of eligible referrals resulting in an IFSP. 
        
    -How do POEs compare on these metrics, and 
    -which POEs are of concern on the basis of one or more of the listed criteria?

In [ ]:
teis_merge2.columns

In [ ]:
teis_merge2.loc[teis_merge2['Referral Age'] < '0 days']
#there is one child whose birthdate must be wrong because they have a negative age at time of referral

In [ ]:
teis_merge2.loc[teis_merge2['Child ID'] == 393629]
#Confirming that child only appears once in the table, so can't correct the birthdate

In [ ]:
teis_merge2 = teis_merge2.loc[teis_merge2['Referral Age'] >= '0 days']
#dropping the instance of a negative age so that it doesn't skew the average for that POE

In [ ]:
teis_merge2['Referral Age'].describe()

In [ ]:
poe_ref_prop = poe_ref
poe_ref_prop['Proportion of Referrals'] = (poe_ref_prop['Total Referrals']/poe_ref_prop['Total Referrals'].sum())*100
poe_ref_prop
#finding what proportion of total referrals each POE handles

It appears that the Northwest and Southwest POEs have particularly low proportions of referrals. I would like to see if there's a way to pull population data in to see if that correlates, though.

In [ ]:
poe_ref_age = teis_merge2.groupby(['POE'])['Referral Age'].mean()
poe_ref_age = poe_ref_age.to_frame().sort_values('Referral Age')
poe_ref_age = poe_ref_age.rename(columns = {'Referral Age' : 'Avg Referral Age in days'})
poe_ref_age['Avg Referral Age in months'] = (poe_ref_age['Avg Referral Age in days']).astype('timedelta64[M]')
poe_ref_age['Difference from Overall Avg in days'] = (poe_ref_age['Avg Referral Age in days']-(teis_merge2['Referral Age'].mean())).astype('timedelta64[D]')
poe_ref_age['Avg Referral Age in days'] = (poe_ref_age['Avg Referral Age in days']).astype('timedelta64[D]')
poe_ref_age

Greater Nashville and South Central may be of concern in this area because there are just over a month and just under two months, respectively, above the overall average age of referral

In [ ]:
poe_suc = ifsp.groupby(['POE'])['Child ID'].count()
poe_suc = poe_suc.to_frame().reset_index()
poe_suc = poe_suc.rename(columns = {'Child ID' : 'Total Successful Referrals'})
poe_suc

In [ ]:
poe_sucrate = pd.merge(poe_ref, poe_suc, how='left').fillna(0)
poe_sucrate['Referral Success Rate'] = (poe_sucrate['Total Successful Referrals'] / poe_ref['Total Referrals'])*100
poe_sucrate

### Bullet 4
Look for patterns in referrals and re-referrals on the basis of the following:

    - Age
    - Referral source
    - Socioeconomic status (County SES)

### Bullet 5
TEIS provides services to eligible infants and toddlers from birth to age 3. 

    -Are there any POE or service coordinators with high rates of early closure (a child exiting before reaching age 3)? 
    -For this question, only consider cases which have reached the IFSP phase.

In [ ]:
ifsp_exit = ifsp.dropna(subset=['Exit Date'])
ifsp_exit.info()

In [ ]:
ifsp_exit = ifsp_exit.loc[ifsp_exit['Referral Age'] > '0 days']
ifsp_exit.info()

In [ ]:
all_early_exits = ifsp_exit.loc[ifsp_exit['Exit Age'] < '1092 days']
all_early_exits
#after discussion and looking at the distribution, we decided to give a 3 day "grace period" prior to the child turning 3

In [ ]:
early_exits = all_early_exits.loc[~all_early_exits['Exit Reason'].isin(['618 - Completion of IFSP/ no longer requires services',
                                                                      '618 - Moved out of state', '618 - Deceased'])]
early_exits
#Pulling out children with these reasons for exit as they would beyond Coordinator's control

In [ ]:
POE_exits = ifsp_exit.groupby('POE')['Child ID'].count()
POE_exits = POE_exits.to_frame().reset_index()
POE_exits = POE_exits.rename(columns = {'Child ID' : 'Total Exits'})
POE_exits
#Total exits by POE

In [ ]:
POE_early_exits = early_exits.groupby('POE')['Child ID'].count()
POE_early_exits = POE_early_exits.to_frame().reset_index()
POE_early_exits = POE_early_exits.rename(columns = {'Child ID' : 'Total Early Exits'})
POE_early_exits
#Total early exits by POE

In [ ]:
POE_early_exit_rate = pd.merge(POE_exits, POE_early_exits)
POE_early_exit_rate['Early Exit Rate'] = round((POE_early_exit_rate['Total Early Exits']/
                                                POE_early_exit_rate['Total Exits'])*100, 2)
POE_early_exit_rate = POE_early_exit_rate.sort_values('Early Exit Rate')
POE_early_exit_rate

In [ ]:
POE_early_exit_rate['Total Early Exits'].sum()/POE_early_exit_rate['Total Exits'].sum()
#Overall Early Exit rate for TEIS

In [ ]:
full_palette = ["#D9ED92", "#B5E48C", "#99D98C", "#76C893", "#52B69A", "#34A0A4", "#168AAD", "#1A759F", "#1E6091", "#184E77"]

main_palette = ["#76C893", "#1E6091"]

In [ ]:
sns.set_theme(style="whitegrid")
eepoe = sns.barplot(x="Early Exit Rate", y="POE", data = POE_early_exit_rate, palette=full_palette)
plt.xlabel("Early Exit Rate")
plt.ylabel("Point of Entry Office")
eepoe.figure.set_size_inches(10, 8)
eepoe.axvline(37.65, linestyle = '--', color='#184E77');
#plt.savefig('../data/eepoe.png', transparent=True)

    The next few cells were just be playing around to how different data looks when filtering for kids who exit more than 90 days before their 3rd birthday

very_early_exits = ifsp_exit.loc[ifsp_exit['Exit Age'] < '1005 days']
very_early_exits

very_early_exits = very_early_exits.loc[~very_early_exits['Exit Reason'].isin(['618 - Completion of IFSP/ no longer requires services',
                                                                      '618 - Moved out of state', '618 - Deceased'])]
very_early_exits

POE_vearly_exits = very_early_exits.groupby('POE')['Child ID'].count()
POE_vearly_exits = POE_vearly_exits.to_frame().reset_index()
POE_vearly_exits = POE_vearly_exits.rename(columns = {'Child ID' : 'Total Very Early Exits'})
POE_vearly_exits

POE_vearly_exit_rate = pd.merge(POE_exits, POE_vearly_exits)
POE_vearly_exit_rate['Very Early Exit Rate'] = round((POE_vearly_exit_rate['Total Very Early Exits']/POE_vearly_exit_rate['Total Exits'])*100, 2)
POE_vearly_exit_rate

In [ ]:
POE_svc_early_exits = early_exits.groupby(['POE', 'Service Coordinator'])['Child ID'].count()
POE_svc_early_exits = POE_svc_early_exits.to_frame().reset_index()
POE_svc_early_exits = POE_svc_early_exits.rename(columns = {'Child ID' : 'Total Early Exits'})
POE_svc_early_exits
#Early exits by POE and Service Coordinator

In [ ]:
POE_svc_exits = ifsp_exit.groupby(['POE', 'Service Coordinator'])['Child ID'].count()
POE_svc_exits = POE_svc_exits.to_frame().reset_index()
POE_svc_exits = POE_svc_exits.rename(columns = {'Child ID' : 'Total Exits'})
POE_svc_exits = POE_svc_exits.loc[POE_svc_exits['Total Exits'] > 3]
POE_svc_exits
#Total exits by POE and Service Coodinator

In [ ]:
POE_svc_exits['Service Coordinator'].value_counts()
#A few coordinators appear under two POE offices

In [ ]:
POE_svc_early_exit_rate = pd.merge(POE_svc_exits, POE_svc_early_exits,  how='left')
POE_svc_early_exit_rate['Early Exit Rate'] = round((POE_svc_early_exit_rate['Total Early Exits']/
                                                POE_svc_early_exit_rate['Total Exits'])*100, 2)
POE_svc_early_exit_rate = POE_svc_early_exit_rate.fillna(0)
POE_svc_early_exit_rate.sort_values('Early Exit Rate', ascending=False)

In [ ]:
UC_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'UC'].nlargest(5, 'Early Exit Rate')
FT_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'FT'].nlargest(5, 'Early Exit Rate')
NW_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'NW'].nlargest(5, 'Early Exit Rate')
ET_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'ET'].nlargest(5, 'Early Exit Rate')
GN_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'GN'].nlargest(5, 'Early Exit Rate')
SW_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'SW'].nlargest(5, 'Early Exit Rate')
SE_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'SE'].nlargest(5, 'Early Exit Rate')
SC_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'SC'].nlargest(5, 'Early Exit Rate')
MD_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'MD'].nlargest(5, 'Early Exit Rate')
POE_svc_high_ee = pd.concat([UC_early_exits, FT_early_exits, NW_early_exits, ET_early_exits, GN_early_exits,
                              SW_early_exits, SE_early_exits, SC_early_exits, MD_early_exits])
POE_svc_high_ee
#Pulling the service coordinators with the highest five individual early exit rates

In [ ]:
sns.set_theme(style="whitegrid")
hee = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = POE_svc_high_ee)
hee.figure.set_size_inches(6, 15)

#### The next 9 visuals are of coordinators with the 5 highest rates for each POE

In [ ]:
sns.set_theme(style="whitegrid")
heeft = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = FT_early_exits, color='#52B69A')
heeft.axvline(38.25, linestyle = '--', color='#184E77')
plt.title('First Tennessee - Overall Rate 38.25%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heeft.set(xlim=(0, 100))
heeft.figure.set_size_inches(6, 4);
#plt.savefig('../data/heeft.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heenw = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = NW_early_exits, color='#1A759F')
heenw.axvline(44.86, linestyle = '--', color='#184E77')
plt.title('Northwest - Overall Rate 44.86%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heenw.set(xlim=(0, 100))
heenw.figure.set_size_inches(6, 4);
#plt.savefig('../data/heenw.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heeet = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = ET_early_exits, color='#168AAD')
heeet.axvline(40.35, linestyle = '--', color='#184E77')
plt.title('East Tennessee - Overall Rate 40.35%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heeet.set(xlim=(0, 100))
heeet.figure.set_size_inches(6, 4);
#plt.savefig('../data/heeet.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heegn = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = GN_early_exits, color='#D9ED92')
heegn.axvline(31.39, linestyle = '--', color='#184E77')
plt.title('Greater Nashville - Overall Rate 31.39%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heegn.set(xlim=(0, 100))
heegn.figure.set_size_inches(6, 4);
#plt.savefig('../data/heegn.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heemd = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = MD_early_exits, color='#76C893')
heemd.axvline(36.83, linestyle = '--', color='#184E77')
plt.title('Memphis Delta - Overall Rate 36.83%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heemd.set(xlim=(0, 100))
heemd.figure.set_size_inches(6, 4);
#plt.savefig('../data/heemd.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heese = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = SE_early_exits, color='#99D98C')
heese.axvline(36.33, linestyle = '--', color='#184E77')
plt.title('Southeast - Overall Rate 36.33%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heese.set(xlim=(0, 100))
heese.figure.set_size_inches(6, 4);
#plt.savefig('../data/heese.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heesc = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = SC_early_exits, color='#B5E48C')
heesc.axvline(33.75, linestyle = '--', color='#184E77')
plt.title('South Central - Overall Rate 33.75%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heesc.set(xlim=(0, 100))
heesc.figure.set_size_inches(6, 4);
#plt.savefig('../data/heesc.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heesw = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = SW_early_exits, color='#1E6091')
heesw.axvline(51.8, linestyle = '--', color='#184E77')
plt.title('Southwest - Overall Rate 51.8%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heesw.set(xlim=(0, 100))
heesw.figure.set_size_inches(6, 4);
#plt.savefig('../data/heesw.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heeuc = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = UC_early_exits, color='#34A0A4')
heeuc.axvline(39.44, linestyle = '--', color='#184E77')
plt.title('Upper Cumberland - Overall Rate 39.44%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heeuc.set(xlim=(0, 100))
heeuc.figure.set_size_inches(6, 4);
#plt.savefig('../data/heeuc.png', bbox_inches = 'tight', transparent=True)

In [ ]:
UC_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'UC') & (POE_svc_early_exit_rate['Early Exit Rate'] > 39.44))]
FT_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'FT') & (POE_svc_early_exit_rate['Early Exit Rate'] > 38.25))]
NW_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'NW') & (POE_svc_early_exit_rate['Early Exit Rate'] > 44.86))]
ET_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'ET') & (POE_svc_early_exit_rate['Early Exit Rate'] > 40.35))]
GN_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'GN') & (POE_svc_early_exit_rate['Early Exit Rate'] > 31.39))]
SW_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'SW') & (POE_svc_early_exit_rate['Early Exit Rate'] > 51.8))]
SE_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'SE') & (POE_svc_early_exit_rate['Early Exit Rate'] > 36.33))]
SC_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'SC') & (POE_svc_early_exit_rate['Early Exit Rate'] > 33.75))]
MD_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'MD') & (POE_svc_early_exit_rate['Early Exit Rate'] > 36.83))]
POE_svc_abavg = pd.concat([UC_aa_early_exits, FT_aa_early_exits, NW_aa_early_exits, ET_aa_early_exits, 
                           GN_aa_early_exits, SW_aa_early_exits, SE_aa_early_exits, SC_aa_early_exits, MD_aa_early_exits])
POE_svc_abavg
#This contais all coordinators whose individual rate is above the rate for their POE

In [ ]:
sns.set_theme(style="whitegrid")
aaee = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = POE_svc_abavg)
aaee.figure.set_size_inches(6, 30)

In [ ]:
POE_abavg_count = POE_svc_abavg.groupby('POE')['Service Coordinator'].count()
POE_abavg_count = POE_abavg_count.to_frame().reset_index()
POE_abavg_count = POE_abavg_count.rename(columns={'Service Coordinator':'Total Service Coordinators w/ high EER'})
POE_abavg_count

In [ ]:
POE_svc_abavg.groupby('POE').describe()

In [ ]:
aaeesvc = sns.FacetGrid(POE_svc_abavg, col="POE", col_wrap=3, hue="POE", palette = 'CMRmap_r')
aaeesvc.map_dataframe(sns.barplot, x="Service Coordinator", y="Early Exit Rate")
aaeesvc.set_axis_labels("Service Coordinator", "Early Exit Rate");

In [ ]:
POE_svc_exits.groupby('POE').describe()

In [ ]:
POE_svc_count = POE_svc_exits.groupby('POE')['Service Coordinator'].count()
POE_svc_count = POE_svc_count.to_frame().reset_index()
POE_svc_count = POE_svc_count.rename(columns={'Service Coordinator':'Total Service Coordinators'})
POE_svc_count

In [ ]:
POE_aasvc_rate = pd.merge(POE_svc_count, POE_abavg_count)
POE_aasvc_rate['Percent of Service Coordinators with high EE rate'] = round((POE_aasvc_rate['Total Service Coordinators w/ high EER']/
                                                POE_aasvc_rate['Total Service Coordinators'])*100, 2)
POE_aasvc_rate = POE_aasvc_rate.sort_values('Percent of Service Coordinators with high EE rate')
POE_aasvc_rate

In [ ]:
POE_aasvc_rate['Total Service Coordinators w/ high EER'].sum() / POE_aasvc_rate['Total Service Coordinators'].sum()
#Overall proportion of service coordinators whose rate is above the average rate for their POE

In [ ]:
UC_early_exits

In [ ]:
GN_early_exits

In [ ]:
SW_early_exits

In [ ]:
NW_early_exits

In [ ]:
ET_early_exits

In [ ]:
FT_early_exits

In [ ]:
MD_early_exits

In [ ]:
SE_early_exits

In [ ]:
SC_early_exits

In [ ]:
for index, row in POE_svc_abavg.iterrows():
    if row['POE'] == "SW":
        POE_svc_abavg.loc[index, 'POE Early Exit Rate'] = '51.80'
    elif row['POE'] == "NW":
        POE_svc_abavg.loc[index, 'POE Early Exit Rate'] = '44.86'
    elif row['POE'] == "ET":
        POE_svc_abavg.loc[index, 'POE Early Exit Rate'] = '40.65'
    elif row['POE'] == "UC":
        POE_svc_abavg.loc[index, 'POE Early Exit Rate'] = '39.44'
    elif row['POE'] == "FT":
        POE_svc_abavg.loc[index, 'POE Early Exit Rate'] = '38.25'
    elif row['POE'] == "MD":
        POE_svc_abavg.loc[index, 'POE Early Exit Rate'] = '36.83'
    elif row['POE'] == "SE":
        POE_svc_abavg.loc[index, 'POE Early Exit Rate'] = '36.33'
    elif row['POE'] == "SC":
        POE_svc_abavg.loc[index, 'POE Early Exit Rate'] = '33.75'
    elif row['POE'] == "GN":
        POE_svc_abavg.loc[index, "POE Early Exit Rate"] = '31.39'
    

In [ ]:
POE_svc_abavg

In [ ]:
ifsp_exit['Exit Year'] = ifsp_exit['Exit Date'].dt.year
ifsp_exit.head()

In [ ]:
poe_exityr = ifsp_exit.groupby(['POE', 'Exit Year'])['Child ID'].count()
poe_exityr = poe_exityr.to_frame().reset_index()
poe_exityr = poe_exityr.rename(columns = {'Child ID' : 'Total Exits'})
poe_exityr

In [ ]:
early_exits['Exit Year'] = early_exits['Exit Date'].dt.year
early_exits.head()

In [ ]:
poe_early_exityr = early_exits.groupby(['POE', 'Exit Year'])['Child ID'].count()
poe_early_exityr = poe_early_exityr.to_frame().reset_index()
poe_early_exityr = poe_early_exityr.rename(columns = {'Child ID' : 'Total Early Exits'})
poe_early_exityr

In [ ]:
POE_yr_early_exit_rate = pd.merge(poe_exityr, poe_early_exityr, how='left')
POE_yr_early_exit_rate['Early Exit Rate'] = round((POE_yr_early_exit_rate['Total Early Exits']/
                                                POE_yr_early_exit_rate['Total Exits'])*100, 2)
POE_yr_early_exit_rate = POE_yr_early_exit_rate.fillna(0)
POE_yr_early_exit_rate

In [ ]:
py = sns.FacetGrid(POE_yr_early_exit_rate, col="POE", col_wrap=3, hue="POE", palette = 'crest')
py.map_dataframe(sns.lineplot, x="Exit Year", y="Early Exit Rate")
py.set_axis_labels("Exit Year", "Early Exit Rate");
#plt.savefig('../data/early_exit_rate_by_poe_+_year', bbox_inches = 'tight', transparent=True)